# Notes for decentmon data processing

In [1]:
# adding required packages if not present
!pip install numpy pandas seaborn matplotlib sklearn

# importing them
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from sklearn.neighbors import KNeighborsClassifier


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Constants
FILENAME = "../decent/log.txt"

In [3]:
# idempotent transformation of input file to csv
# file = open(FILENAME, "rt")
# raw_data = file.read()
# raw_data = raw_data.replace(' ', ',') #add commas to read it as a csv
# file.close()
# file = open(FILENAME, "wt")
# file.write(raw_data)
# file.close()

In [4]:
df = pd.read_csv(FILENAME, sep='@')
df.insert(0, 'formula_id', range(0, len(df)))
df

,formula_id,x,cent_trace_len,decent_trace_len,odecent_trace_len,cent_num_mess,decent_num_mess,odecent_num_mess,cent_size_mess,decent_size_mess,odecent_size_mess,cent_nb_progressions,decent_nb_progressions,odecent_nb_progressions,formula,trace
0,0,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""b"")",{a|b| } ; { | | } ; {a| | } ; { | |c} ; { |b|...
1,1,1,2,3,2,3,2,0,1.0,1.66,0.00,4,4,4,"Ev (Var ""b"")",{ | |c} ; {a|b|c} ; {a| |c} ; {a|b|c} ; { |b|...
2,2,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""a"")",{ |b|c} ; {a| | } ; {a| |c} ; { | | } ; {a|b|...
3,3,1,6,7,6,15,2,0,1.0,0.55,0.00,12,24,12,"Glob (Var ""b"")",{a|b| } ; {a|b|c} ; { |b| } ; { |b|c} ; {a|b|...
4,4,1,2,3,2,3,2,0,1.0,1.66,0.00,4,8,4,"Glob (Var ""c"")",{a| |c} ; { | | } ; { | | } ; {a| | } ; {a| |...
5,5,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""b"")",{a|b|c} ; { |b| } ; { |b| } ; {a| |c} ; { | |...
6,6,1,4,5,4,9,2,0,1.0,0.83,0.00,8,12,8,"Ev (Var ""c"")",{a| | } ; { |b| } ; {a|b| } ; {a| |c} ; { | |...
7,7,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""a"")",{a|b|c} ; {a| |c} ; {a| | } ; {a|b|c} ; {a| |...
8,8,1,4,5,4,9,2,0,1.0,0.83,0.00,8,16,8,"Glob (Var ""b"")",{ |b| } ; { |b| } ; {a|b| } ; {a| |c} ; { |b|...
9,9,1,2,3,2,3,0,0,1.0,0.00,0.00,3,3,3,"Next (Var ""c"")",{a|b| } ; {a| |c} ; {a|b|c} ; {a|b| } ; {a|b|...


In [5]:
def split(data):
    """
    Splits the multiple-strategies Decentmon dataframe in 3 single-strategy dataframes
    :param data: 3-strategy dataframe
    :return: triple of single-strategy dataframes
    """
    def filter_clean(prefix, replacement):
        to_return = data.filter(regex=f"^{prefix}.*").copy()
        to_return['formula_id'] = data['formula_id']
        to_return.columns = to_return.columns.str.replace(prefix, "")
        to_return['strategy'] = replacement
        return to_return
    return filter_clean("cent_", "orchestration"), filter_clean("decent_", "migration"), filter_clean("odecent_", "choreography")

(cent, decent, odecent ) = split(df)

merged = pd.concat([cent, decent, odecent], ignore_index=True)
merged

,formula_id,strategy
0,0,orchestration
1,1,orchestration
2,2,orchestration
3,3,orchestration
4,4,orchestration
...,...,...
76,22,choreography
77,23,choreography
78,24,choreography
79,25,choreography


In [6]:
def extract_metric(data, metric):
    """
    Extracts the given column metric to the pair of columns (metric, value)
    :param data: the dataframe having the current metric
    :param metric: the metric to extract
    :return: a copy of the original dataframe where the metric column is replaced with the pair (metric, value)
    """
    data = merged[['formula_id', 'strategy', metric]].copy()
    data['metric'] = metric
    data = data.rename(columns={metric:"value"})
    return data

trace_length = extract_metric(merged, "trace_len")
num_mess = extract_metric(merged, "num_mess")
size_mess = extract_metric(merged, "size_mess")
nb_progressions = extract_metric(merged, "nb_progressions")
merged_metrics = pd.concat([trace_length, num_mess, size_mess, nb_progressions])
merged_metrics

KeyError: "['trace_len'] not in index"

In [ ]:
def plot(data, ax=None):
    """
    Plots the data from a single-strategy dataframe
    :param data: single-strategy dataframe
    :return: nothing. Plots the data
    """
    g = sns.catplot(
        #ax=ax,
        data=data, kind="bar",
        ci="sd", palette="dark", alpha=.8, height=6
    )
    g.set_axis_labels("", "Count")

In [ ]:
# Plotting
plot(merged)

In [ ]:
g = sns.FacetGrid(merged_metrics, col="metric", aspect=1.5, sharey=False)
g.map(sns.barplot, "strategy", "value", order=["orchestration", "migration", "choreography"])

In [ ]:
ndf = merged_metrics.copy()
ndf['strategy'] = ndf['strategy'].str.replace("^cent_", "1")
ndf['strategy'] = ndf['strategy'].str.replace("^decent_", "2")
ndf['strategy'] = ndf['strategy'].str.replace("^odecent_", "3")
ndf

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(ndf, 'strategy')
